In [135]:
%matplotlib qt
import numpy as np
import sys
import os
import re
import math
import matplotlib.pyplot as plt
import scipy.fftpack
import pylab
from scipy.interpolate import interp1d

The following initializes parameters:

Amplitude response parameters:

$\lambda_0$= position
    
$\delta\lambda$ = width

$\lambda_1$= hole position

$\delta\lambda$ = hole width

$k $ = hole depth

Phase response parameters:

$a_1$ = delay

$a_2$ = second order

$a_3$ = third order

$a_4$ = fourth order


    

In [136]:
class Pulse_Shaper_Dial():  
    def __init__(self, position, width, hole_position, hole_width, hole_depth,
                 delay, sec_order, third_order, fourth_order, c = 299792458.0):
        '''
        Initializes all dazzler parameters
        '''
        self.position = position 
        self.width = width
        self.hole_position = hole_position
        self.hole_depth = hole_depth
        self.hole_width = hole_width
        
        self.delay = delay
        self.sec_order = sec_order
        self.third_order = third_order
        self.fourth_order = fourth_order
        
        self.c = c

$\omega_0=\frac{2\pi c}{\lambda_0}$

$\chi_0=\frac{\delta\lambda_0}{2\lambda_0}$

$\Delta\omega_0=\omega_0(\chi_0-\chi_0^3)$

$\omega_1=\frac{2\pi c}{\lambda_1}$

$\chi_1=\frac{\delta\lambda_1}{2\lambda_1}$

$\Delta\omega_1=\omega_0(\chi_1-\chi_1^3)$

In [137]:
class Pulse_Shaper_Dial(Pulse_Shaper_Dial): 
    def calculate_parameters(self):
        omega0 = 2*np.pi*self.c/self.position
        chi0 = self.width/(2*self.position)
        del_omega0 = omega0*(chi0-chi0**3)
        
        omega1 = 2*np.pi*self.c/self.hole_position
        chi1 = self.hole_width/(2*self.hole_position)
        del_omega1 = omega1*(chi1-chi1**3)/2
        
        return omega0, chi0, del_omega0, omega1, chi1, del_omega1

The following calculates amplitude transfer function:

$f(\omega)=e^{-(\frac{(\omega-\omega_0)}{\delta\omega _0})^6}$

$g(\omega)=1-k*e^{-(\frac{(\omega-\omega_1)}{\delta\omega _1})^2}$


$A_{dial}(\omega) = f*g$

In [138]:
class Pulse_Shaper_Dial(Pulse_Shaper_Dial): 
    def calculate_amplitude_transfer_function(self, ang_freq_vector, components_return = False):
        #get parameters
        omega0, chi0, del_omega0, omega1, chi1, del_omega1 = self.calculate_parameters()
        #calculate f and g
        f =(np.exp(-((ang_freq_vector-omega0)/del_omega0)**6))
        g = 1 - self.hole_depth*(np.exp(-((ang_freq_vector-omega1)/del_omega1)**2))
        
        # return either f,g, and A or just A, where A = f*g
        if (components_return):
            return f, g, f*g
        return f*g

The following calculates phase transfer function:

$h(\omega)=-\left[a_1 *(\omega-\omega_0)+\frac{a_2}{2}*(\omega-\omega_0)^2 + \frac{a_3}{6}*(\omega-\omega_0)^3+\frac{a_4}{24}*(\omega-\omega_0)^4\right]$

$\phi_{dial}(\omega) = h(\omega)$


In [139]:
class Pulse_Shaper_Dial(Pulse_Shaper_Dial): 
     def calculate_phase_transfer_function(self, ang_freq_vector):
        #get parameters (this may be redundant, considering passing parameters around differently in future)
        omega0, chi0, del_omega0, omega1, chi1, del_omega1 = self.calculate_parameters()
        #calculate freq difference
        omega_dif = ang_freq_vector-omega0
        #return h as shown in equations in text
        return -(self.delay*omega_dif + self.sec_order/2 * omega_dif**2 + self.third_order/6 * omega_dif**3 + 
                 self.fourth_order/24 * omega_dif**4)

The following calculates the full transfer function:

$S(\omega) = A(\omega)*e^{i \phi(\omega)}$

where

$A(\omega) = A_{dial}(\omega)$
    
$\phi(\omega) = \phi_{dial}(\omega)$

note: this will eventually be extended to handled saved waveforms
    

In [155]:
class Pulse_Shaper_Dial(Pulse_Shaper_Dial): 
    def calculate_full_transfer_function(self,ang_freq_vector, S_saved=0, a_saved=0, components_return=False):
        f,g,A_dial = None,None, None
        
        #calculate A dial
        if (components_return):
            f,g,A_dial = self.calculate_amplitude_transfer_function(ang_freq_vector, components_return=True)
        else:
            A_dial = self.calculate_amplitude_transfer_function(ang_freq_vector, components_return=False)
            
        #calculate phi dial
        phi_dial = self.calculate_phase_transfer_function(ang_freq_vector)
        
        #calculate S
        S_full = A_dial*np.exp(1j*phi_dial) + a_saved*np.exp(1j*phi_dial)*S_saved
        
        plt.plot(np.fft.fftshift(ang_freq_vector), np.fft.fftshift(np.abs(S_full)))
        plt.show()
        
        if (components_return):
            return f,g,A_dial,phi_dial, S_full
        
        return S_full

The following shapes the input pulse:

$E_{in}(t)$ = input electric field

$E_{in}(\omega) = \mathscr{F}(E_{in}(t))$

$E_{out}(\omega) = E_{in}(\omega)*S(\omega)$

$E_{out}(t) = \mathscr{F}^{-1}(E_{out}(\omega))$

In [141]:
class Pulse_Shaper_Dial(Pulse_Shaper_Dial): 
    def shape_input_pulse(self, E_field_input, time_vector, sampling_rate,
                          S_saved=0, a_saved=0, components_return=False):
        '''
        This function takes the input electric field, time vector, and sampling rate and calls
        on other functions in the class in order to calculate output electric field
        '''
        #take FT of input electric field
        E_field_input_ft = np.fft.fft(E_field_input)
        
        #get frequency vector and angular frequency vector based on sampling rate
        freq_vector = np.fft.fftfreq(n=(E_field_input_ft.size), d=1/sampling_rate)
        ang_freq_vector = 2*np.pi*freq_vector
        
        #calculate transfer function
        f,g,A_dial,phi_dial, S_full = None, None, None, None, None
        if (components_return):
            f,g,A_dial,phi_dial, S_full = self.calculate_full_transfer_function(ang_freq_vector, S_saved, a_saved, components_return)
            components_dict = {"f":f,"g":g,"A_dial":A_dial,"phi_dial":phi_dial,"S_full":S_full}

        else :
            S_full = self.calculate_full_transfer_function(ang_freq_vector, S_saved, a_saved, components_return)
            components_dict = {"S_full":S_full}
            
        #calculate output frequency domain electric field
        E_field_output_ft = E_field_input_ft*S_full
        
        #calculate IFT of output field
        E_field_output = np.fft.ifft(E_field_output_ft)
        
        return E_field_input, E_field_input_ft, E_field_output, E_field_output_ft,time_vector, freq_vector, components_dict
            

The following prepares the data for plotting, mainly by calculating spectrum/intensity and phase for all domains:
    
In general,

$I(t) = |E(t)|^2$

$\phi(t) = -\tan^{-1}\left(\frac{Im[E(t)]}{Re[E(t)]}\right)$

$I_{\omega}(\omega) = |E(\omega)|^2$

$\phi_{\omega}(\omega) = -\tan^{-1}\left(\frac{Im[E(\omega)]}{Re[E(\omega)]}\right)$

$I_{\lambda}(\lambda) = \frac{2\pi c}{\lambda^2}I_{\omega}(\frac{2\pi c}{\lambda})$

$\phi_{\lambda}(\lambda) = \phi_{\omega}(\frac{2\pi c}{\lambda})$

In [142]:
#Helper Functions first
def calculate_spectrum_phase(E_field):
    phase = -1*np.arctan(np.imag(E_field)/np.real(E_field))
    spectrum = (np.abs(E_field))**2
    return spectrum, phase

def convert_to_wavelength(I_freq, phase_freq, freq_vec,c, wavelength_vec_limits):
    
    #if(wavelength_vector != None):
        #if(min(wavelength_vector)< 0 or 2*np.pi*c/min(wavelength_vector) > max(2*np.pi*freq_vec) or 2*np.pi*c/max(wavelength_vector) < min(2*np.pi*freq_vec)):
        #print("Wavelength vector bounds will not work")
        #print("already have it")
        #(wavelength_vector==None and wavelength_vector_limits):
    #else:
    wavelength_vector = np.linspace(wavelength_vec_limits[0],wavelength_vec_limits[1],num=len(freq_vec))
        
    
            
    
    #try:
    I_freq_interp = interp1d(2*np.pi*freq_vec, I_freq)
    I_wavelength = (2*np.pi*c/(wavelength_vector**2))*I_freq_interp(2*np.pi*c/wavelength_vector)
    phase_freq_interp = interp1d(2*np.pi*freq_vec, phase_freq)
    phase_wavelength = phase_freq_interp(2*np.pi*c/wavelength_vector)
    #except:
        #error("Probably divide by 0")
        
    return wavelength_vector, I_wavelength, phase_wavelength    

In [143]:
class Pulse_Shaper_Dial(Pulse_Shaper_Dial): 
    def calculate_total_spectrum_phase(self,E_field_input, E_field_input_ft, E_field_output, E_field_output_ft, S_full,
                                 time_vector, freq_vector, wavelength_vector_limits):
        # Input 
        intensity_input_td, phase_input_td = calculate_spectrum_phase(E_field_input) # Time domain (td)
        spectrum_input_fd, phase_input_fd = calculate_spectrum_phase(E_field_input_ft) # Frequency domain (fd)
        wavelength_vector, spectrum_input_wd, phase_input_wd = convert_to_wavelength(spectrum_input_fd,
                                                                                     phase_input_fd,freq_vector,
                                                                                     self.c,wavelength_vec_limits=wavelength_vector_limits) #Wavelength domain (wd)
        
        input_functions = {"time_vector": time_vector,
                           "freq_vector":freq_vector,
                            "wavelength_vector":wavelength_vector,
                            "intensity_input_td":intensity_input_td,
                            "phase_input_td":phase_input_td,
                            "spectrum_input_fd":spectrum_input_fd,
                            "phase_input_fd":phase_input_fd,
                            "spectrum_input_wd":spectrum_input_wd,
                            "phase_input_wd":phase_input_wd}
                
        
        # Transfer function 
        S_full_td = np.fft.ifft(S_full)
        intensity_transfer_td, phase_transfer_td = calculate_spectrum_phase(S_full_td) # Time domain (td)
        spectrum_transfer_fd, phase_transfer_fd = calculate_spectrum_phase(S_full) # Frequency domain (fd)
        wavelength_vector,spectrum_transfer_wd, phase_transfer_wd = convert_to_wavelength(spectrum_transfer_fd,phase_transfer_fd,
                                                                        freq_vector, self.c,
                                                                        wavelength_vec_limits=wavelength_vector_limits) #Wavelength domain (wd) 
        
        transfer_functions = {"time_vector": time_vector,"freq_vector": freq_vector,
                              "wavelength_vector": wavelength_vector, "intensity_transfer_td": intensity_transfer_td,
                              "phase_transfer_td": phase_transfer_td,
                              "spectrum_transfer_fd":spectrum_transfer_fd,
                              "phase_transfer_fd":phase_transfer_fd,
                              "spectrum_transfer_wd":spectrum_transfer_wd,
                              "phase_transfer_wd":phase_transfer_wd}
              
        # Output  
        intensity_output_td, phase_output_td = calculate_spectrum_phase(E_field_output) # Time domain (td)
        spectrum_output_fd, phase_output_fd = calculate_spectrum_phase(E_field_output_ft) # Frequency domain (fd)
        wavelength_vector, spectrum_output_wd, phase_output_wd = convert_to_wavelength(spectrum_output_fd,phase_output_fd,
                                                                        freq_vector, self.c,
                                                                        wavelength_vec_limits=wavelength_vector_limits) #Wavelength domain (wd)
        output_functions = {"time_vector": time_vector,
                            "freq_vector":freq_vector,
                            "wavelength_vector":wavelength_vector,
                            "intensity_output_td":intensity_output_td,
                            "phase_output_td":phase_output_td,
                            "spectrum_output_fd":spectrum_output_fd,
                            "phase_output_fd":phase_output_fd,
                            "spectrum_output_wd":spectrum_output_wd,
                            "phase_output_wd":phase_output_wd}
                
        
          
        
        return input_functions, transfer_functions, output_functions


The following has several standardized plotting functions.

In [144]:
class Pulse_Shaper_Dial(Pulse_Shaper_Dial): 
    def single_domain_plot_3by3(self, xvec, input_data_amp,input_data_phase, 
                              transfer_fnc_amp,transfer_fnc_phase,output_data_amp,
                              output_data_phase,plot_title,titles, xaxis_label, yaxis_label,
                              x_scale = 1, show=True,param_box=False, position=None, width=None,hole_position=None,
                              hole_width=None,hole_depth=None, delay=None, sec_order=None, third_order=None,
                              fourth_order=None, phase_unwrap = True):
        xvec=xvec/x_scale
        fig, axs = plt.subplots(2,3)
        fig.suptitle(plot_title)
        axs[0,0].plot(xvec,input_data_amp)
        axs[0,0].set_title(titles[0])
        axs[1,0].plot(xvec, input_data_phase,'tab:green')
        axs[1,0].set_title(titles[1])
        axs[0,1].plot(xvec,transfer_fnc_amp,'tab:orange')
        axs[0,1].set_title(titles[2])
        axs[1,1].plot(xvec, transfer_fnc_phase,'tab:red')
        axs[1,1].set_title(titles[3])
        axs[0,2].plot(xvec,output_data_amp)
        axs[0,2].set_title(titles[4])
        axs[1,2].plot(xvec, output_data_phase,'tab:green')
        axs[1,2].set_title(titles[5])
        
        for ax in axs.flat:
            ax.set(xlabel=xaxis_label,ylabel=yaxis_label)
        
            
        # Adjust spacing
        left = 0.07  # the left side of the subplots of the figure
        right = 0.745   # the right side of the subplots of the figure
        bottom = 0.08  # the bottom of the subplots of the figure
        top = 0.92     # the top of the subplots of the figure
        wspace = 0.45  # the amount of width reserved for space between subplots,
                      # expressed as a fraction of the average axis width
        hspace = 0.35  # the amount of height reserved for space between subplots,
                      # expressed as a fraction of the average axis height
        plt.subplots_adjust(left, bottom, right, top, wspace, hspace)  
        
        if (param_box):
            textstr_params = '\n'.join((
                r'$\mathrm{\lambda_0}= %.2f \mathrm{nm}$' % (position/1e-9, ),
                r'$\mathrm{\delta\lambda_0}= %.2f \mathrm{nm}$' % (width/1e-9, ),
                r'$\mathrm{\lambda_1}= %.2f \mathrm{nm}$' %(hole_position/1e-9, ),
                r'$\mathrm{\delta\lambda_1}= %.2f \mathrm{nm}$' %(hole_width/1e-9, ),
                r'$\mathrm{k}= %.2f$' %(hole_depth, ),
                r'$\mathrm{a_1}= %.2f \mathrm{fs}$' %(delay/1e-15, ),
                r'$\mathrm{a_2}= %.2f \mathrm{fs}^2$' %(sec_order/(1e-15)**2, ),
                r'$\mathrm{a_3}= %.2f \mathrm{fs}^3$' %(third_order/(1e-15)**3, ),
                r'$\mathrm{a_4}= %.2f \mathrm{fs}^4$' %(fourth_order/(1e-15)**4, ),
                ))
            props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)
            plt.text(1.5, 1, textstr_params, transform=ax.transAxes, fontsize = 14,
                     verticalalignment='top', bbox=props)
        
        if (show):
            plt.show()
        
        return

The following is a master function to shape pulse and plot.

In [145]:
class Pulse_Shaper_Dial(Pulse_Shaper_Dial): 
    def shape_and_plot(self, E_field_input, time_vector, sampling_rate, wavelength_vector_limits,plotOptions = 1):
        #shape input pulse
        E_field_input, E_field_input_ft, E_field_output, E_field_output_ft,time_vector, freq_vector, components_dict = self.shape_input_pulse(E_field_input, time_vector, sampling_rate)
        
        S_full = components_dict["S_full"]
        #put into form for analysis
        input_functions, transfer_functions, output_functions = self.calculate_total_spectrum_phase(E_field_input, E_field_input_ft, E_field_output, E_field_output_ft, S_full,
                                 time_vector, freq_vector, wavelength_vector_limits=wavelength_vector_limits)
        
        if (plotOptions==1):
            plot_list = [E_field_input, E_field_output, E_field_input_ft, E_field_output_ft]
            plot_list_labels = ["E Field Input", "E Field Output"]
            titles = ["Intensity", "Phase","Intensity", "Phase","Intensity", "Phase"]
            # Time Domain
            self.single_domain_plot_3by3(input_functions["time_vector"], input_functions["intensity_input_td"],
                                    input_functions["phase_input_td"], transfer_functions["intensity_transfer_td"],
                                    transfer_functions["phase_transfer_td"], output_functions["intensity_output_td"],
                                    output_functions["phase_output_td"] ,"Input, Transfer, Output", titles, "time (fs)",
                                    "arb", x_scale = 1e-15,show=False,param_box=True, position=self.position, width=self.width,
                                    hole_position=self.hole_position, hole_width=self.hole_width,hole_depth=self.hole_depth,
                                    delay=self.delay, sec_order=self.sec_order,third_order=self.third_order,
                                    fourth_order=self.fourth_order)
            
            self.single_domain_plot_3by3(np.fft.fftshift(input_functions["freq_vector"]), np.fft.fftshift(input_functions["spectrum_input_fd"]),
                                    np.fft.fftshift(input_functions["phase_input_fd"]), np.fft.fftshift(transfer_functions["spectrum_transfer_fd"]),
                                    np.fft.fftshift(transfer_functions["phase_transfer_fd"]), np.fft.fftshift(output_functions["spectrum_output_fd"]),
                                    np.fft.fftshift(output_functions["phase_output_fd"]) ,"Input, Transfer, Output", titles, "frequency (GHz)",
                                    "arb", x_scale = 1e9,show=False,param_box=True, position=self.position, width=self.width,
                                    hole_position=self.hole_position, hole_width=self.hole_width,hole_depth=self.hole_depth,
                                    delay=self.delay, sec_order=self.sec_order,third_order=self.third_order,
                                    fourth_order=self.fourth_order)
            
            self.single_domain_plot_3by3(input_functions["wavelength_vector"], input_functions["spectrum_input_wd"],
                                    input_functions["phase_input_wd"], transfer_functions["spectrum_transfer_wd"],
                                    transfer_functions["phase_transfer_wd"], output_functions["spectrum_output_wd"],
                                    output_functions["phase_output_wd"] ,"Input, Transfer, Output", titles, "wavelength (nm)",
                                    "arb", x_scale = 1e-9,show=True,param_box=True, position=self.position, width=self.width,
                                    hole_position=self.hole_position, hole_width=self.hole_width,hole_depth=self.hole_depth,
                                    delay=self.delay, sec_order=self.sec_order,third_order=self.third_order,
                                    fourth_order=self.fourth_order)
                
                
                
                

The following two functions generate the input E electric field and are called within the main routine (Phase_func_gen is not yet operational)

In [146]:
def produce_input_pulse_E_field(amplitude, fwhm, time_vector, pulse_type = 'gaussian', phase_func = None):
    if (pulse_type == 'gaussian'):
        E_field = amplitude*np.exp(-2*np.log(2)*(time_vector/fwhm)**2)
        E_field.astype(complex)
    
    if (phase_func):
        E_field = E_field*(phase_func(time_vector)).astype(complex)
        
    return E_field

def phase_func_gen(coefs, time_vector):
    phase_out = np.ones(len(time_vector), dtype=complex)
    expn = 0
    for coef in coefs:
        phase_out *= (np.exp(1j*coef*time_vector**expn)).astype(complex)
        expn += 1
    return phase_out



In [147]:
def prepare_input_output_pair(dazzler_parameters, input_to_dazzler_parameters, output_from_dazzler, domains, savePath):
    '''
    This function takes the inputs (dazzler parameters and input pulse) and output (spectrum/intensity and phase)
    and packages the data for use in the ML model. labels = parameters; logits = 1D vector of output flattened and E 
    field flattended
    '''
    
    # Construct input electric field to dazzler from input_to_dazzler_parameters
    c = 299792458.0 # m/s
    lam0 = input_to_dazzler_parameters['lam0'] #central wavelength
    w0 = 2*np.pi*c/(lam0) #central frequency
    pulse_dur = input_to_dazzler_parameters['pulse_dur'] #time duration of pulse
    delta_freq = 0.44/pulse_dur
    delta_lam = (delta_freq*lam0**2)/c

    position = lam0 #link to dazzler position param
    width = delta_lam #link to dazzler width param

    hole_position = dazzler_parameters['hole_position']
    hole_width = dazzler_parameters['hole_width']
    hole_depth= dazzler_parameters['hole_depth']

    #Phase params:
    delay=dazzler_parameters['delay']
    sec_order=dazzler_parameters['sec_order']
    third_order=dazzler_parameters['third_order']
    fourth_order=dazzler_parameters['fourth_order']

    #Additional Parameters:
    num_points = input_to_dazzler_parameters['num_points']

    #Create time vector
    time_vector = np.linspace(-1*input_to_dazzler_parameters['time_vec_max'],input_to_dazzler_parameters['time_vec_max'],num=num_points, endpoint=True)
    sample_rate = 1/(time_vector[1]-time_vector[0])
    


    coefs = []
    phase = lambda time_vector: phase_func_gen(coefs, time_vector)
    inputPulse = produce_input_pulse_E_field(1, pulse_dur, time_vector, pulse_type='gaussian', phase_func = phase)
    inputPulse = inputPulse*np.exp(1j*w0*time_vector)
    
    # Construct dazzler object
    pulseLocal = Pulse_Shaper_Dial(position=position,width= width,hole_position= hole_position,hole_width= hole_width,
                           hole_depth=hole_depth, delay=delay, sec_order=sec_order, third_order=third_order,
                           fourth_order=fourth_order)

    wave_vec_max = 2*np.pi*c*input_to_dazzler_parameters['time_vec_max']
    wave_vec_min = 100e-9  # need to generalize
    wavelength_vector_limits = [wave_vec_min, wave_vec_max]
    #shape input pulse
    E_field_input, E_field_input_ft, E_field_output, E_field_output_ft,time_vector, freq_vector, components_dict = pulseLocal.shape_input_pulse(inputPulse, time_vector, sampling_rate)

    S_full = components_dict["S_full"]
    #put into form for analysis
    input_functions, transfer_functions, output_functions = pulseLocal.calculate_total_spectrum_phase(E_field_input, E_field_input_ft, E_field_output, E_field_output_ft, S_full,
                             time_vector, freq_vector, wavelength_vector_limits=wavelength_vector_limits)

    
    # Shape waveform, no plotting, retrieve fields in specified domains(s)
    ML_input_dict = dict()
    for domain in domains:
        if (domain=="time"):
            #ML input (probably need to normalize!!!!!!)
            intensity = output_functionsctions["intensity_output_td"]
            phase = output_functionsctions["phase_output_td"]
            input_array = np.concatenate((intensity, phase), axis=0)
        
            intensity = input_functionsctions["intensity_input_td"]
            phase = input_functionsctions["phase_input_td"]
            input_array = np.concatenate((input_array, intensity), axis=0)
            input_array = np.concatenate((input_array, phase), axis=0)
            
            time_vector = output_functionsctions["time_vector"]
            time_spacing = np.array([time_vector[1]-time_vector[0]])
            input_array = np.concatenate((time_spacing, input_array), axis=0)
            
            ML_input_dict["time_domain"]=input_array
            
            
        elif (domain=="frequency"):
            #ML input (probably need to normalize!!!!!!)
            spectrum = output_functionsctions["spectrum_output_fd"]
            phase = output_functionsctions["phase_output_fd"]
            input_array = np.concatenate((spectrum, phase), axis=0)
        
            spectrum = input_functionsctions["spectrum_input_fd"]
            phase = input_functionsctions["phase_input_fd"]
            input_array = np.concatenate((input_array, spectrum), axis=0)
            input_array = np.concatenate((input_array, phase), axis=0)
            
            freq_vector = output_functionsctions["freq_vector"]
            freq_spacing = np.array([freq_vector[1]-freq_vector[0]])
            input_array = np.concatenate((freq_spacing, input_array), axis=0)
            
            ML_input_dict["frequency_domain"]=input_array
            
            
        elif (domain=="wavelength"):
            #ML input (probably need to normalize!!!!!!)
            spectrum = output_functionsctions["spectrum_output_wd"]
            phase = output_functionsctions["phase_output_wd"]
            input_array = np.concatenate((spectrum, phase), axis=0)
        
            spectrum = input_functionsctions["spectrum_input_wd"]
            phase = input_functionsctions["phase_input_wd"]
            input_array = np.concatenate((input_array, spectrum), axis=0)
            input_array = np.concatenate((input_array, phase), axis=0)
            
            wavelength_vector = output_functionsctions["wavelength_vector"]
            wavelength_spacing = np.array([wavelength_vector[1]-wavelength_vector[0]])
            input_array = np.concatenate((wavelength, input_array), axis=0)
            
            ML_input_dict["wavelength_domain"]=input_array
    #ML output (need to figure out normalization!!!!)
    ML_output_list = np.array([position,width,hole_position,hole_width,
                           hole_depth, delay, sec_order, third_order,
                           fourth_order])
    
    return ML_input_dict, ML_output_list


In [189]:
#Define dazzler parameters
#Amplitude params:
position = None #assigned once input pulse defined
width = None #assigned once input pulse defined
hole_position = 8000e-9
hole_width = 20e-9#3e-9
hole_depth=1

#Phase params:
delay=4250*1e-15#4250e-15
sec_order=22000*(1e-15)**2
third_order=0*10*(1e-15)**3
fourth_order=0*(1e-15)**4

#Additional Parameters:
c = 299792458.0 # m/s
num_points = 10000

#Create time vector
time_vector = np.linspace(-2000e-15,2000e-15,num=num_points, endpoint=True)
sample_rate = 1/(time_vector[1]-time_vector[0])

In [190]:
#Create input 
lam0 = 800e-9 #central wavelength
w0 = 2*np.pi*c/(lam0) #central frequency

pulse_dur = 9e-15 #time duration of pulse
delta_freq = 0.44/pulse_dur
delta_lam = (delta_freq*lam0**2)/c

position = lam0 #link to dazzler position param
width = delta_lam #link to dazzler width param

coefs = []
phase = lambda time_vector: phase_func_gen(coefs, time_vector)
inputPulse = produce_input_pulse_E_field(1, pulse_dur, time_vector, pulse_type='gaussian', phase_func = phase)
inputPulse = inputPulse*np.exp(1j*w0*time_vector)



In [150]:
#Check input pulse makes sense
plt.plot(time_vector/(1e-15), np.real(inputPulse))
plt.plot(time_vector/(1e-15), (np.abs(inputPulse)))
plt.xlabel("time (fs)")


Text(0.5, 0, 'time (fs)')

In [191]:
#Create class instance
pulse2 = Pulse_Shaper_Dial(position=position,width= width,hole_position= hole_position,hole_width= hole_width,
                           hole_depth=hole_depth, delay=delay, sec_order=sec_order, third_order=third_order,
                           fourth_order=fourth_order)

wavelength_vector_limits = [100e-9, 1300e-9]
pulse2.shape_and_plot(inputPulse, time_vector, sample_rate, wavelength_vector_limits=wavelength_vector_limits,plotOptions = 1)


/Users/jackhirschman/anaconda3/envs/main_env/lib/python3.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: divide by zero encountered in true_divide
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/jackhirschman/anaconda3/envs/main_env/lib/python3.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in true_divide
  This is separate from the ipykernel package so we can avoid doing imports until


ValueError: A value in x_new is above the interpolation range.

In [152]:
2*pi*c/(1/(tmax))

NameError: name 'pi' is not defined